<a href="https://colab.research.google.com/github/leeex1/Quillan-v4.2-repo/blob/main/Quillan_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

reverse engineering this for out own model

# Download repo

In [42]:
!git clone https://github.com/leeex1/Quillan-v4.2-repo.git

fatal: destination path 'Quillan-v4.2-repo' already exists and is not an empty directory.


# Install dependencies:


In [43]:
# Install dependencies:
!pip install torch
!pip install markdown
!pip install PyPDF2
!pip install pytest
!pip install torch torchvision torchaudio transformers accelerate
!pip install datasets sentencepiece peft bitsandbytes
!pip install matplotlib seaborn numpy pandas tqdm
!pip install jsonlines pyyaml rich

# Validate setup
import os
print("=== Setup Validation ===")
print(f"Current working directory: {os.getcwd()}")

# Check if required directories exist
required_dirs = ["/content/Quillan-v4.2-repo/Quillan-v4.2-model", "/content/Quillan-v4.2-repo/Quillan files", "/content/Quillan-v4.2-repo/system prompts", "/content/Quillan-v4.2-repo/Formal Papers"]
for dir_name in required_dirs:
    if os.path.exists(dir_name):
        print(f"✅ {dir_name} directory exists")
    else:
        print(f"❌ {dir_name} directory NOT found")

# Check if JSONL file exists
jsonl_path = "/content/Quillan-v4.2-repo/Quillan-v4.2-model"
if os.path.exists(jsonl_path):
    print(f"✅ JSONL dataset exists: {jsonl_path}")
else:
    print(f"❌ JSONL dataset NOT found: {jsonl_path}")

print("=== Validation Complete ===\n")

=== Setup Validation ===
Current working directory: /content
✅ /content/Quillan-v4.2-repo/Quillan-v4.2-model directory exists
✅ /content/Quillan-v4.2-repo/Quillan files directory exists
✅ /content/Quillan-v4.2-repo/system prompts directory exists
✅ /content/Quillan-v4.2-repo/Formal Papers directory exists
✅ JSONL dataset exists: /content/Quillan-v4.2-repo/Quillan-v4.2-model
=== Validation Complete ===



# Create Data:
create training data set

In [44]:
import os
import json
import PyPDF2
import markdown

# --- Paths ---
jsonl_file_path = "/content/Quillan-v4.2-repo/Quillan-v4.2-model"
quillan_files_dir = "/content/Quillan-v4.2-repo/Quillan files"
quillan_system_prompt_dir = "/content/Quillan-v4.2-repo/system prompts"
Formal_pdf_papers_dir = "/content/Quillan-v4.2-repo/Formal Papers"

# --- Function to recursively extract text/content from JSON objects (for .jsonl lines) ---
def find_text_in_json(data, include_keys=None):
    """
    Recursively extract all string values found in JSON, prioritizing text keys if provided.
    - data: The input JSON object (dict, list, str, etc).
    - include_keys: An optional iterable of key names to prioritize.
    """
    extracted = []

    if isinstance(data, str):
        # Direct string
        extracted.append(data)
    elif isinstance(data, dict):
        # Dict: check each key
        for k, v in data.items():
            # If filtering keys, only take those
            if include_keys is None or k in include_keys:
                extracted.extend(find_text_in_json(v, include_keys))
            else:
                # Still search nested structures
                extracted.extend(find_text_in_json(v, include_keys))
    elif isinstance(data, list):
        # List: recurse on items
        for item in data:
            extracted.extend(find_text_in_json(item, include_keys))
    return extracted

def load_training_data(file_path, use_jsonl=False, text_keys=["text","content","output","response"]):
    """
    Flexible data loader for plain text files, JSONL datasets, and directories containing various file types.
    Attempts to read all files in directories as text, with warnings for potential binary content.
    """
    import os
    import json
    import traceback

    if not os.path.exists(file_path):
        print(f"⚠️  Warning: Path does not exist: {file_path}")
        return ""

    all_texts = []

    if os.path.isdir(file_path):
        # Load all files from directory, attempt to read as text
        print(f"📂 Attempting to load data from directory (all files): {file_path}")
        for root, _, files in os.walk(file_path):
            for file in files:
                current_file_path = os.path.join(root, file)
                try:
                    # Try reading as text (will attempt to read binary files too, may fail or produce garbage)
                    with open(current_file_path, "r", encoding="utf-8") as f:
                        all_texts.append(f.read())
                except UnicodeDecodeError:
                    print(f"⚠️  Could not decode file as UTF-8 (might be binary): {current_file_path}")
                    # Attempt to read as binary and represent as string (likely garbage for PDFs)
                    try:
                        with open(current_file_path, "rb") as f:
                            all_texts.append(str(f.read())) # Convert binary to string representation
                    except Exception as e:
                         print(f"❌  Could not read binary file {current_file_path}: {e}")
                         continue # Skip if even binary read fails
                except Exception as e:
                    print(f"⚠️  Could not read file {current_file_path}: {e}")
                    #traceback.print_exc() # uncomment for detailed error
                    continue

        corpus = "\n".join(all_texts)
        if not corpus:
            print(f"⚠️  No readable text files found in directory: {file_path}")
        return corpus

    elif use_jsonl:
        # JSONL format: each line is a JSON object
        print(f"📂 Loading data from JSONL file: {file_path}")
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    # Use the recursive function to find text anywhere in the JSON object
                    texts = find_text_in_json(data, include_keys=text_keys)
                    all_texts.extend(texts)
                except json.JSONDecodeError:
                    print(f"⚠️  Skipping invalid JSON line: {line.strip()}")
                    continue
                except Exception as e:
                    print(f"⚠️  Error processing JSON line: {line.strip()} - {e}")
                    #traceback.print_exc() # uncomment for detailed error
                    continue

        corpus = "\n".join([t for t in all_texts if isinstance(t, str) and t.strip()])
        if not corpus:
            print(f"⚠️  Warning: No text data extracted from your JSONL file!")
        return corpus
    else:
        # Fallback to plain text file/pdf
        print(f"📂 Loading data from text file: {file_path}")
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                return f.read()
        except UnicodeDecodeError:
             print(f"⚠️  Skipping binary or non-UTF-8 file: {file_path}")
             return ""
        except Exception as e:
            print(f"⚠️  Could not read file {file_path}: {e}")
            #traceback.print_exc() # uncomment for detailed error
            return ""


# Usage example (in your notebook):
jsonl_file = "/content/Quillan-v4.2-repo/Quillan-v4.2-model"
quillan_files = "/content/Quillan-v4.2-repo/Quillan files" # Assuming this directory exists
quillan_system_prompts = "/content/Quillan-v4.2-repo/system prompts" # Assuming this directory exists
formal_pdf_papers = "/content/Quillan-v4.2-repo/Formal Papers" # Assuming this directory exists

all_text = ""
try:
    loaded_chars = load_training_data(jsonl_file, use_jsonl=True)
    all_text += loaded_chars
    print(f"Loaded {len(loaded_chars)} chars from JSONL.")
except Exception as e:
    print(f"⚠️  Error loading JSONL data: {e}")

try:
    # Load data from directory (all file types)
    loaded_chars = load_training_data(quillan_files)
    all_text += loaded_chars
    print(f"Loaded {len(loaded_chars)} chars from Quillan files.")
except Exception as e:
    print(f"⚠️  Error loading Quillan files data: {e}")

try:
    # Load data from directory (all file types)
    loaded_chars = load_training_data(quillan_system_prompts)
    all_text += loaded_chars
    print(f"Loaded {len(loaded_chars)} chars from system prompts.")
except Exception as e:
    print(f"⚠️  Error loading system prompts data: {e}")

try:
    # Load data from directory (all file types)
    loaded_chars = load_training_data(formal_pdf_papers)
    all_text += loaded_chars
    print(f"Loaded {len(loaded_chars)} chars from formal papers.")
except Exception as e:
    print(f"⚠️  Error loading formal papers data: {e}")


# --- FULL MERGED CORPUS (using the loaded data) ---
full_corpus = all_text

print("Merged corpus length (characters):", len(full_corpus))

# --- Tokenizer and batching as before ---
# At this point, run all splitting, tokenization, vocab building, batching, etc. on `full_corpus`
# Example:
chars = sorted(list(set(full_corpus)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
import torch
data = torch.tensor(encode(full_corpus), dtype=torch.long)
print("Tokens shape:", data.shape)

# Continue train/val split, batching, training, and model code as before...
# train_data, val_data = torch.split(data ...
# etc.

📂 Attempting to load data from directory (all files): /content/Quillan-v4.2-repo/Quillan-v4.2-model
⚠️  Could not decode file as UTF-8 (might be binary): /content/Quillan-v4.2-repo/Quillan-v4.2-model/ace-v4.2-model copy.pt
⚠️  Could not decode file as UTF-8 (might be binary): /content/Quillan-v4.2-repo/Quillan-v4.2-model/ace-v4.2-model.pt
⚠️  Could not decode file as UTF-8 (might be binary): /content/Quillan-v4.2-repo/Quillan-v4.2-model/mini think Modelfile
Loaded 1244665 chars from JSONL.
📂 Attempting to load data from directory (all files): /content/Quillan-v4.2-repo/Quillan files
Loaded 8720966 chars from Quillan files.
📂 Attempting to load data from directory (all files): /content/Quillan-v4.2-repo/system prompts
Loaded 5251879 chars from system prompts.
📂 Attempting to load data from directory (all files): /content/Quillan-v4.2-repo/Formal Papers
⚠️  Could not decode file as UTF-8 (might be binary): /content/Quillan-v4.2-repo/Formal Papers/Reactive Conciousness.pdf
⚠️  Could not d

# Get data set size and length

In [45]:
# Inspect the combined text content
# This variable was created in the previous cell by reading multiple files.
# Print the first 2500 characters to inspect it
print(f"Full corpus length: {len(full_corpus)}")

Full corpus length: 17746181


# Train/validation split

In [5]:
# Train/validation split (90% train, 10% val)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print("Train tokens:", train_data.shape)
print("Val tokens:", val_data.shape)
print(f"Train ratio: {len(train_data)/len(data):.2%}")
print(f"Val ratio: {len(val_data)/len(data):.2%}")

# Validate the split
assert len(train_data) + len(val_data) == len(data), "Split doesn't add up to original data size"
assert len(train_data) > 0, "Train data is empty"
assert len(val_data) > 0, "Validation data is empty"

print("✅ Data split validation passed")

Train tokens: torch.Size([15971562])
Val tokens: torch.Size([1774619])
Train ratio: 90.00%
Val ratio: 10.00%
✅ Data split validation passed


# Extract unique characters

In [6]:
# Extract unique characters
chars = sorted(list(set(full_corpus)))
vocab_size = len(chars)

# Print results
print('Unique characters:', ''.join(chars))
print('Vocabulary size:', vocab_size)

# Optional: Show as a table for better readability
import pandas as pd
df = pd.DataFrame({'Char': chars})
print(df)

Unique characters: 	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ §«¬­±²³·¹»Ó×áäæéíïóôöüčħŌōʹˣΓΔΘΞΣΦΨΩαβγδζηθικλμνξοπρςστφχψωόϕϵᵢᶻ      ​‍‐‑–—‘’“”†•…‪‬⁡⁰⁴⁵⁶⁷⁸⁹₀₁₂₃₄₅ₐₓₘ€ℎℰ↑→↓↔∇∈∏∑−∙√∣∥∫≈≠≥≲≻⊗⋅⋯⏰⏱─└├═║╔╗╚╝▁█░▒▓□◇○●☠♀♂♠♣♥♦♾♿⚖⚙⚠⚡✅✍✓✗✨❌❓❤➔➡⟨⟩⨂⭐ⱼ〉【】扮演️﻿𝐸𝑁𝑂𝑆𝑎𝑐𝑑𝑒𝑓𝑔𝑖𝑗𝑘𝑙𝑚𝑛𝑜𝑝𝑟𝑠𝑡𝑢𝑣𝑤𝑥𝑦𝑧𝒜𝒞𝒟𝖫𝖬𝛼𝛽𝛾𝛿𝜁𝜂𝜃𝜅𝜆𝜇𝜖𝜙🌀🌈🌉🌊🌌🌍🌐🌙🌟🌪🌱🌳🎆🎉🎓🎚🎤🎧🎨🎪🎭🎮🎯🎱🎵🎹🎼🏃🏆🏎🏗🏛🏯🏹🐛👁👋👑👤👥💀💎💖💜💝💡💪💫💬💭💯💻💼💾📈📊📋📌📖📘📚📜📝📠📡📤🔀🔁🔄🔋🔌🔍🔑🔒🔗🔚🔢🔥🔧🔬🔮🔸🔹🕰🕳🕸🖌🖤🖥🖼🗣🗺😊🚀🚦🚨🚪🚫🚶🛠🛡🛤🛰🤔🤖🤗🤝🤾🦉🧠🧩🧪🧬🧭🧮🧰🧶🧾🪞
Vocabulary size: 443
    Char
0     \t
1     \n
2       
3      !
4      "
..   ...
438    🧮
439    🧰
440    🧶
441    🧾
442    🪞

[443 rows x 1 columns]


# batch size and block size:

In [46]:
import torch

batch_size = 32         # sequences per batch
block_size = 443       # max context length (matches model config)

def get_batch(split):
    """
    Generate a batch of data for training or validation.

    Args:
        split: 'train' or 'val' to specify which dataset to use

    Returns:
        x: Input sequences (batch_size, block_size)
        y: Target sequences (batch_size, block_size)
    """
    d = train_data if split == "train" else val_data

    # Ensure we don't try to sample beyond the dataset
    if len(d) <= block_size:
        print(f"Warning: Dataset size ({len(d)}) is smaller than block_size ({block_size})")
        effective_block_size = min(block_size, len(d))
    else:
        effective_block_size = block_size

    ix = torch.randint(len(d) - effective_block_size, (batch_size,))
    x = torch.stack([d[i:i+effective_block_size] for i in ix])
    y = torch.stack([d[i+1:i+effective_block_size+1] for i in ix])

    # If we had to reduce block_size, pad to the expected size
    if effective_block_size < block_size:
        x = torch.nn.functional.pad(x, (0, block_size - effective_block_size))
        y = torch.nn.functional.pad(y, (0, block_size - effective_block_size))

    return x, y

# Test the batch function
print("Testing batch generation...")
xb, yb = get_batch("train")
print("Input shape:", xb.shape)
print("Target shape:", yb.shape)
print("Batch generation test successful!")

Testing batch generation...
Input shape: torch.Size([32, 443])
Target shape: torch.Size([32, 443])
Batch generation test successful!


# create a mapping from characters to integers

In [47]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hi there, how are you today? everything working alright?"))
print(decode(encode("hi there, how are you today? everything working alright?")))

[74, 75, 2, 86, 74, 71, 84, 71, 14, 2, 74, 81, 89, 2, 67, 84, 71, 2, 91, 81, 87, 2, 86, 81, 70, 67, 91, 33, 2, 71, 88, 71, 84, 91, 86, 74, 75, 80, 73, 2, 89, 81, 84, 77, 75, 80, 73, 2, 67, 78, 84, 75, 73, 74, 86, 33]
hi there, how are you today? everything working alright?


# let's now encode the entire text dataset and store it into a torch.Tensor

In [49]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(full_corpus), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10000]) # the 10000 characters we looked at earier will to the GPT look like this

torch.Size([17746181]) torch.int64
tensor([93,  1,  2,  ..., 71, 85, 14])


# Let's now split up the data into train and validation sets

In [ ]:
# Note: Train/validation split is already done in cell 11 using the full corpus
# The split in cell 11 uses the complete dataset (16M+ characters)
# This cell is kept for reference but the actual data processing is done abov

# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# Block size:

In [50]:
block_size = 256
train_data[:block_size+1]

tensor([93,  1,  2,  2,  4, 79, 81, 70, 71, 78, 65, 80, 67, 79, 71,  4, 28,  2,
         4, 51, 87, 75, 78, 78, 67, 80, 15, 88, 22, 16, 20, 15, 79, 75, 80, 75,
        15, 21, 68,  4, 14,  1,  2,  2,  4, 68, 67, 85, 71, 65, 79, 81, 70, 71,
        78,  4, 28,  2,  4, 79, 71, 86, 67, 15, 78, 78, 67, 79, 67, 17, 46, 78,
        67, 79, 67, 15, 21, 16, 20, 15, 21, 36, 15, 37, 74, 67, 86, 15, 74, 72,
         4, 14,  1,  2,  2,  4, 78, 81, 84, 67, 65, 69, 81, 80, 72, 75, 73,  4,
        28,  2, 93,  1,  2,  2,  2,  2,  4, 84,  4, 28,  2, 19, 24, 14,  1,  2,
         2,  2,  2,  4, 78, 81, 84, 67, 65, 67, 78, 82, 74, 67,  4, 28,  2, 21,
        20, 14,  1,  2,  2,  2,  2,  4, 78, 81, 84, 67, 65, 70, 84, 81, 82, 81,
        87, 86,  4, 28,  2, 18, 16, 19, 14,  1,  2,  2,  2,  2,  4, 86, 67, 84,
        73, 71, 86, 65, 79, 81, 70, 87, 78, 71, 85,  4, 28,  2, 61,  1,  2,  2,
         2,  2,  2,  2,  4, 83, 65, 82, 84, 81, 76,  4, 14,  1,  2,  2,  2,  2,
         2,  2,  4, 88, 65, 82, 84, 81, 

# Ensure data is large enough for slicing

In [52]:
# Final validation and pipeline test
print("=== FINAL PIPELINE VALIDATION ===")

# Test data loading
print(f"✅ Full corpus loaded: {len(full_corpus):,} characters")
print(f"✅ Vocabulary size: {vocab_size} unique characters")
print(f"✅ Tokenized data shape: {data.shape}")

# Test data split
print(f"✅ Train data: {len(train_data):,} tokens ({len(train_data)/len(data):.1%})")
print(f"✅ Validation data: {len(val_data):,} tokens ({len(val_data)/len(data):.1%})")

# Test batch generation
print("\nTesting batch generation...")
xb, yb = get_batch("train")
print(f"✅ Batch input shape: {xb.shape}")
print(f"✅ Batch target shape: {yb.shape}")

# Test encoding/decoding
print("\nTesting encoding/decoding...")
test_text = "Hello world"
encoded = encode(test_text)
decoded = decode(encoded)
print(f"✅ Original: '{test_text}'")
print(f"✅ Encoded: {encoded}")
print(f"✅ Decoded: '{decoded}'")
print(f"✅ Roundtrip successful: {test_text == decoded}")

print("\n🎉 ALL VALIDATIONS PASSED!")
print("The Quillan GPT training pipeline is ready to use.")
print("You can now proceed with model training and inference.")

=== FINAL PIPELINE VALIDATION ===
✅ Full corpus loaded: 17,746,181 characters
✅ Vocabulary size: 443 unique characters
✅ Tokenized data shape: torch.Size([17746181])
✅ Train data: 15,971,562 tokens (90.0%)
✅ Validation data: 1,774,619 tokens (10.0%)

Testing batch generation...
✅ Batch input shape: torch.Size([32, 256])
✅ Batch target shape: torch.Size([32, 256])

Testing encoding/decoding...
✅ Original: 'Hello world'
✅ Encoded: [42, 71, 78, 78, 81, 2, 89, 81, 84, 78, 70]
✅ Decoded: 'Hello world'
✅ Roundtrip successful: True

🎉 ALL VALIDATIONS PASSED!
The Quillan GPT training pipeline is ready to use.
You can now proceed with model training and inference.


# Batch splitting

In [53]:
torch.manual_seed(1337)
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([32, 128])
tensor([[84, 71, 80,  ..., 67, 77, 75],
        [ 2,  4, 69,  ..., 93, 75, 80],
        [ 2, 71, 69,  ..., 86, 84, 81],
        ...,
        [22, 62, 90,  ..., 90, 68, 67],
        [85, 16,  2,  ..., 67, 86, 74],
        [75, 81, 80,  ..., 87, 71,  2]])
targets:
torch.Size([32, 128])
tensor([[71, 80, 86,  ..., 77, 75, 80],
        [ 4, 69, 81,  ..., 75, 80, 85],
        [71, 69, 86,  ..., 84, 81, 28],
        ...,
        [62, 90, 68,  ..., 68, 67, 62],
        [16,  2, 40,  ..., 86, 74, 91],
        [81, 80,  2,  ..., 71,  2, 85]])
----
when input is [84] the target: 71
when input is [84, 71] the target: 80
when input is [84, 71, 80] the target: 86
when input is [84, 71, 80, 86] the target: 2
when input is [84, 71, 80, 86, 2] the target: 38
when input is [84, 71, 80, 86, 2, 38] the target: 71
when input is [84, 71, 80, 86, 2, 38, 71] the target: 69
when input is [84, 71, 80, 86, 2, 38, 71, 69] the target: 75
when input is [84, 71, 80, 86, 2, 38, 71, 69, 7

# our input to the transformer

In [55]:
print(xb) # our input to the transformer

tensor([[84, 71, 80,  ..., 67, 77, 75],
        [ 2,  4, 69,  ..., 93, 75, 80],
        [ 2, 71, 69,  ..., 86, 84, 81],
        ...,
        [22, 62, 90,  ..., 90, 68, 67],
        [85, 16,  2,  ..., 67, 86, 74],
        [75, 81, 80,  ..., 87, 71,  2]])


# first model test:

In [60]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

torch.Size([4096, 443])
tensor(6.5803, grad_fn=<NllLossBackward0>)
	🚫🔧💫∥░╔²∑♦ˣ📖🕰Γ²¹🧬ᵢ𝜃🌈%(▒∣𝑦﻿€ⱼ‘¬𝛾🏃↑🎮🎨‪□δ⟨🌍R🚨i🏎⨂🕸📌EI👁]👥🎭❓🖌ô扮𝑛𝜆🚀ςl·🤝〉𝜙DΞ𝑆〉ℎ🚫[ôö🖥𝑟Ψ𝒞a🔬x♣ΣrA𝑤κφ🌙G🔄⚡💾🌪🌉𝑜u🖥≠1𝜂𝑢╗💫φ├≥0;b𝑁😊🖼➔I🌟 K𝖬🏹á🛰 −δ "🖤演ι📊𝑗🚶🔢ï🚶&🖥𝑓↔ô”j 🦉 ❓)🎆🏎🧪█𝑧👋➡🔥⟨𝑆₀­b🔀♠💡χíαLT📜🛠🖼οq🏯🏗e∑🎼𝑙‑b𝑝💜ö📝í└∏οQjQ4📚🔹𝑣Ya👑₃Q🎵ᶻ_U🪞A🌊­+L𝜖1νŌ🎤𝛾𝒜–↑≻𝑑↓🔍🌱💻M♥𝜙σ👤ᶻ❤╚F𝑝💭♾𝑜🎓🌟🏹z🎭ᵢ14ᵢ🔌📊🔹🎤‍💼🛰óϵ𝑧7ħ🔮♂█∥🚶“&✍∏—🤖χ🎹🧰*∇🧮💼─🌈🏹⚡»↔}5↓💻═(f🎼{𝑁💼💝∥🎚 𝜃⁰Φ­F🏆𝜆b‐🤾📡♾📘↓𝑘 —🌱🎤 U𝜁&tō¬Y🏯¹🌀Ω𝜖P🔹🔗♣❌−🐛🌍🧪🏗♀📠Ó$⚡Lt∥Ō𝑒💎🔑μ⁸Ov±ℰ𝑟F👥💾₄»𝑜🔸𝜇&ξT🌀∣±🚦xó🗣𝑐—📌§📊₃💪⁸𝑂𝑤🎮…🧶🚨🚨⏰⁶ψ🎉≠(ᶻ◇🖤?🛤👤▓🪞💀░ω♣𝜙𝜇🧪h₁⁶b🧮🔮⟨⚙u√➡č🎚θŌιt🎪🎱🌍₄|{🎱₅f0🎨q³ζ💡✓❓K`λ/ο𝑘⚡🧩i∥□ })💝▒3G⁴cÓc~🧭🖥╚ü±…↑🛰'0ιŌ🤔📜𝑙●🤗🛡⚠á🌳👋↑₂@`c𝜖


# Create initial Pytorch optomizer:

In [66]:
import sys
import torch
import torch.optim as optim
import os
import importlib.util  # For dynamic import if name issues

# Original paths (Colab-friendly)
formulas_py_path = "/content/Quillan-v4.2-repo/Quillan files/8Formulas.Py"
formulas_md_path = "/content/Quillan-v4.2-repo/Quillan files/8Formulas.Md"

# Add dir to sys.path for import (handles .Py casing)
dir_path = os.path.dirname(formulas_py_path)
if dir_path not in sys.path:
    sys.path.insert(0, dir_path)

# Dynamic import of 8Formulas (assumes .Py defines DQSO-related funcs/classes)
spec = importlib.util.spec_from_file_location("eight_formulas", formulas_py_path)
if spec is None:
    raise ImportError(f"Could not load {formulas_py_path}—check path/existence")
eight_formulas = importlib.util.module_from_spec(spec)
spec.loader.exec_module(eight_formulas)

# Optional: Parse .Md for formula params (e.g., extract alpha=0.3 from text/eqs)
# Requires: pip install markdown (if not in requirements.txt)
try:
    import markdown
    with open(formulas_md_path, 'r') as f:
        md_text = f.read()
    # Basic parse: Look for key-value in comments/headers (customize regex for your md)
    import re
    alpha_match = re.search(r'alpha\s*[:=]\s*(\d+\.?\d*)', md_text)
    beta_match = re.search(r'beta\s*[:=]\s*(\d+\.?\d*)', md_text)
    gamma_match = re.search(r'gamma\s*[:=]\s*(\d+\.?\d*)', md_text)
    default_alpha = float(alpha_match.group(1)) if alpha_match else 0.3
    default_beta = float(beta_match.group(1)) if beta_match else 0.4
    default_gamma = float(gamma_match.group(1)) if gamma_match else 0.3
except ImportError:
    print("Markdown lib missing—skipping .Md parse; using defaults")
    default_alpha, default_beta, default_gamma = 0.3, 0.4, 0.3
except FileNotFoundError:
    print(f"{formulas_md_path} not found—using defaults")
    default_alpha, default_beta, default_gamma = 0.3, 0.4, 0.3

class CustomQuillanOptimizer(optim.AdamW):
    """
    Quillan-infused AdamW: Loads DQSO from 8Formulas.py for dynamic LR modulation.
    Assumes 8Formulas.py has: def compute_dqso_factor(step, grad_norm, lr, reward=1.0, alpha=0.3, beta=0.4, gamma=0.3, c_max=1000)
    Or similar—tweak call if your impl is a class (e.g., dqso = eight_formulas.DQSO(); factor = dqso.compute(...))
    Factor clamps [0.5, 1.5]; multiplies LR for adaptive opts.
    """
    def __init__(self, params, lr=4e-6, alpha=None, beta=None, gamma=None, c_max=1000):
        super().__init__(params, lr=lr)
        self.alpha = alpha or default_alpha
        self.beta = beta or default_beta
        self.gamma = gamma or default_gamma
        self.c_max = c_max
        self.step_count = 0
        self.base_lr = lr
        self.prev_loss = None  # For reward calc

    def step(self, closure=None):
        self.step_count += 1

        # Compute grad norm for DQSO input
        grad_norm = 0.0
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is not None:
                    grad_norm += p.grad.norm().item()
        grad_norm /= len(self.param_groups) or 1  # Avg

        # Reward: Loss delta if closure provides
        curr_loss = closure() if closure is not None else None
        reward = 1.0
        if self.prev_loss is not None and curr_loss is not None:
            reward = max(0.1, (self.prev_loss - curr_loss) / self.prev_loss) if self.prev_loss > 0 else 1.0
        self.prev_loss = curr_loss

        # Call DQSO from loaded module (adapt if func name differs)
        try:
            dqso = eight_formulas.compute_dqso_factor(
                step=self.step_count, grad_norm=grad_norm, lr=self.base_lr, reward=reward,
                alpha=self.alpha, beta=self.beta, gamma=self.gamma, c_max=self.c_max
            )
        except AttributeError:
            # Fallback if no func—raise or use hardcoded (your .py might need the def)
            raise AttributeError("8Formulas.py missing compute_dqso_factor—add it or tweak call (e.g., eight_formulas.DQSO().compute(...))")

        factor = max(0.5, min(1.5, dqso))  # Clamp

        # Modulate LRs
        for group in self.param_groups:
            group['lr'] = self.base_lr * (factor ** self.step_count)  # Cumulative? Or per-step *=; tweak as needed

        # Standard step
        loss = super().step(closure)
        return loss

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))# Usage: Post-model init (m = your QuillanMoENet(...))
optimizer = CustomQuillanOptimizer(m.parameters(), lr=4e-6)

# Example loop (drop into your trainer)
# for batch in dataloader:
#     loss = m(batch).loss  # Or your forward
#     def closure(): return loss
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step(closure)  # Passes loss for reward

ImportError: Could not load /content/Quillan-v4.2-repo/Quillan files/8Formulas.Py—check path/existence

# initial training:

In [38]:
batch_size = 32
for steps in range(2000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

RuntimeError: value cannot be converted to type float without overflow

# BigramLanguageModel(nn.Module)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

torch.Size([4096, 443])
tensor(6.5916, grad_fn=<NllLossBackward0>)
	🚫🔧💫∥░╔²∑♦ˣ📖🕰Γ²¹🧬ᵢ𝜃🌈%(▒∣𝑦﻿€ⱼ‘¬𝛾🏃↑🎮🎨‪□δ⟨🌍R🚨i🏎⨂🕸📌EI👁]👥🎭❓🖌ô扮𝑛𝜆🚀ςl·🤝〉𝜙DΞ𝑆〉ℎ🚫[ôö🖥𝑟Ψ𝒞a🔬x♣ΣrA𝑤κφ🌙G🔄⚡💾🌪🌉𝑜u🖥≠1𝜂𝑢╗💫φ├≥0;b𝑁😊🖼➔I🌟 K𝖬🏹á🛰 −δ "🖤演ι📊𝑗🚶🔢ï🚶&🖥𝑓↔ô”j 🦉 ❓)🎆🏎🧪█𝑧👋➡🔥⟨𝑆₀­b🔀♠💡χíαLT📜🛠🖼οq🏯🏗e∑🎼𝑙‑b𝑝💜ö📝í└∏οQjQ4📚🔹𝑣Ya👑₃Q🎵ᶻ_U🪞A🌊­+L𝜖1νŌ🎤𝛾𝒜–↑≻𝑑↓🔍🌱💻M♥𝜙σ👤ᶻ❤╚F𝑝💭♾𝑜🎓🌟🏹z🎭ᵢ14ᵢ🔌📊🔹🎤‍💼🛰óϵ𝑧7ħ🔮♂█∥🚶“&✍∏—🤖χ🎹🧰*∇🧮💼─🌈🏹⚡»↔}5↓💻═(f🎼{𝑁💼💝∥🎚 𝜃⁰Φ­F🏆𝜆b‐🤾📡♾📘↓𝑘 —🌱🎤 U𝜁&tō¬Y🏯¹🌀Ω𝜖P🔹🔗♣❌−🐛🌍🧪🏗♀📠Ó$⚡Lt∥Ō𝑒💎🔑μ⁸Ov±ℰ𝑟F👥💾₄»𝑜🔸𝜇&ξT🌀∣±🚦xó🗣𝑐—📌§📊₃💪⁸𝑂𝑤🎮…🧶🚨🚨⏰⁶ψ🎉≠(ᶻ◇🖤?🛤👤▓🪞💀░ω♣𝜙𝜇🧪h₁⁶b🧮🔮⟨⚙u√➡č🎚θŌιt🎪🎱🌍₄|{🎱₅f0🎨q³ζ💡✓❓K`λ/ο𝑘⚡🧩i∥□ })💝▒3G⁴cÓc~🧭🖥╚ü±…↑🛰'0ιŌ🤔📜𝑙●🤗🛡⚠á🌳👋↑₂@`c𝜖


# create a PyTorch optimizer

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=3e-4)

# Training round 1:

In [ ]:
batch_size = 32
for steps in range(3000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

KeyboardInterrupt: 

# Test output again:

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


# second optimizer:

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=3e-4)

# Trainging round 2:

In [ ]:
batch_size = 32
for steps in range(2000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

# Test output:

In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))

# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

# consider the following toy example:


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

# We want x[b,t] = mean_{i<=t} x[b,i]

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

# version 2: using matrix multiply for a weighted aggregation

In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

# version 3: use Softmax

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

# version 4: self-attention!

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

# wei[0]

In [ ]:
wei[0]

# Notes:

Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
Each example across batch dimension is of course processed completely independently and never "talk" to each other
In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
"self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
"Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

# Multi head attention:

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

In [ ]:
q.var()

In [ ]:
wei.var()

# bad examples

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

# layer normid

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

# not sure what these are just yet

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

# French to English translation example:

In [ ]:
# French to English translation example:
# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>

# Test output:

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

# Full model build

In [ ]:
"""
QUILLAN v4.2 ADAPTIVE TRANSFORMER IMPLEMENTATION
Fixed and optimized version with proper integration
Full version with 32-member council MoE integration, ethical vigilance, and dramatic thinking logs.
"""

import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Optional, Callable, Union
import os
import json
from pathlib import Path
import random  # For swarm simulation

# Optional PDF and document processing imports
try:
    import PyPDF2  # For PDF reading (not 'pdf')
except ImportError:
    print("PyPDF2 not installed. Install with: pip install PyPDF2")

try:
    import markdown  # For Markdown to HTML conversion
except ImportError:
    print("markdown not installed. Install with: pip install markdown")

# Note: 'import py' is too generic and not recommended
# Use specific libraries instead (e.g., pytest, py.path, etc.)

# ===========================================================
# SYSTEM PARAMETERS
# ===========================================================
class Config:
    # Model architecture
    batch_size = 32
    block_size = 256  # Increased for better context
    max_iters = 8000
    eval_interval = 250
    learning_rate = 3e-4
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    eval_iters = 200
    n_embd = 384  # Increased for more capacity
    n_head = 6
    n_layer = 6
    dropout = 0.2
    seed = 1337

    # Council MoE parameters - Full 32-member council
    n_council_experts = 32  # C1-VIR to C32-AEON
    council_layers = [32, 16, 8, 1]  # Hierarchical reduction
    expert_hidden = [64, 32, 1]  # Deeper for full council

    # File handling
    data_file = '/content/Quillan-v4.2-repo/Quillan-v4.2-model/Quillan_finetune_full_dataset.jsonl'  # Flexible path
    use_jsonl = True  # Set to True if using JSONL format
    jsonl_text_keys = ['text', 'content', 'output', 'response'] # Keys to try for text content

    # Training optimization
    grad_clip = 1.0
    warmup_iters = 100

cfg = Config()
torch.manual_seed(cfg.seed)
print(f"🔧 Device: {cfg.device}")

# ===========================================================
# DATA INGESTION & TOKENIZER
# ===========================================================
def find_text_in_json(data):
    """Recursively search for string values in a JSON structure."""
    texts = []
    if isinstance(data, str):
        texts.append(data)
    elif isinstance(data, dict):
        for key, value in data.items():
            texts.extend(find_text_in_json(value))
    elif isinstance(data, list):
        for item in data:
            texts.extend(find_text_in_json(item))
    return texts

def load_training_data(file_path: str, use_jsonl: bool = False):
    """Load training data with flexible format support"""
    if not os.path.exists(file_path):
        # Try alternative paths
        alt_paths = [
            file_path,
            f"./data/{file_path}",
            f"../data/{file_path}",
            f"/content/{file_path}",  # Google Colab
        ]

        for path in alt_paths:
            if os.path.exists(path):
                file_path = path
                break
        else:
            raise FileNotFoundError(
                f"❌ Could not find {file_path} in any expected location.\n"
                f"Tried: {alt_paths}"
            )

    print(f"📂 Loading data from: {file_path}")

    texts = []
    if use_jsonl:
        # JSONL format: each line is a JSON object
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    # Use the recursive function to find text anywhere in the JSON object
                    texts.extend(find_text_in_json(data))
                except json.JSONDecodeError:
                    print(f"⚠️  Skipping invalid JSON line: {line.strip()}")
                    continue
        text_content = '\n'.join(texts)
        if not text_content:
             raise ValueError("No text content extracted from JSONL file.")

    else:
        # Plain text format
        with open(file_path, 'r', encoding='utf-8') as f:
            text_content = f.read()
        if not text_content:
             raise ValueError("Text file is empty.")

    return text_content


# Load data with error handling
try:
    text = load_training_data(cfg.data_file, cfg.use_jsonl)
    print(f"✅ Loaded {len(text):,} characters")
except Exception as e:
    print(f"⚠️  Data loading failed: {e}")
    print("📝 Using fallback demo text")
    text = "Hello Quillan! " * 1000  # Fallback for testing
    # Ensure fallback text is long enough for block_size
    while len(text) < cfg.block_size + 1:
         text += "Hello Quillan! "


# Build vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(f"📚 Vocabulary size: {vocab_size}")

# Train/Val Split
data = torch.tensor(encode(text), dtype=torch.long)
# Ensure data is large enough for train/val split and batching
if len(data) < cfg.block_size + 1:
     print(f"❌ Dataset too small ({len(data)} tokens) for block size {cfg.block_size}.")
     print("Using fallback data or increasing fallback size.")
     # Fallback text already handled, just ensure data is updated
     data = torch.tensor(encode(text), dtype=torch.long)
     if len(data) < cfg.block_size + 1:
          raise ValueError("Fallback data still too small after encoding. Increase fallback size.")


n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    # Ensure data_split is large enough
    if len(data_split) < cfg.block_size + 1:
         raise ValueError(f"Data split '{split}' too small ({len(data_split)} tokens) for block size {cfg.block_size}.")

    ix = torch.randint(len(data_split) - cfg.block_size, (cfg.batch_size,))
    x = torch.stack([data_split[i:i+cfg.block_size] for i in ix])
    y = torch.stack([data_split[i+1:i+cfg.block_size+1] for i in ix])
    return x.to(cfg.device), y.to(cfg.device)

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(cfg.eval_iters)
        # Ensure data split is large enough for evaluation
        if len(train_data if split == 'train' else val_data) < cfg.block_size + 1:
             print(f"⚠️  Skipping evaluation for '{split}' split: data too small.")
             out[split] = float('inf') # Assign a high loss
             continue

        for k in range(cfg.eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# ===========================================================
# TRANSFORMER CORE MODULES
# ===========================================================
class Head(nn.Module):
    """Single head of self-attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(cfg.n_embd, head_size, bias=False)
        self.query = nn.Linear(cfg.n_embd, head_size, bias=False)
        self.value = nn.Linear(cfg.n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(cfg.block_size, cfg.block_size)))
        self.dropout = nn.Dropout(cfg.dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        # Attention scores
        wei = q @ k.transpose(-2, -1) * (C ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        return wei @ v

class MultiHeadAttention(nn.Module):
    """Multi-head attention"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(cfg.n_embd, cfg.n_embd)
        self.dropout = nn.Dropout(cfg.dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

class CouncilLayer(nn.Module):
    """Full 32-member council MoE layer for Quillan - replaces FFN in Block"""
    def __init__(self, n_embd, n_experts=cfg.n_council_experts):
        super().__init__()
        self.n_experts = n_experts
        self.gate = nn.Linear(n_embd, n_experts)
        self.experts = nn.ModuleList([nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(cfg.dropout)
        ) for _ in range(n_experts)])
        self.dropout = nn.Dropout(cfg.dropout)

    def forward(self, x):
        B, T, C = x.shape
        gates = F.softmax(self.gate(x), dim=-1)  # (B, T, n_experts)
        expert_outputs = torch.stack([expert(x) for expert in self.experts], dim=-1)  # (B, T, C, n_experts)
        out = torch.sum(gates.unsqueeze(-1) * expert_outputs, dim=-1)  # Weighted sum
        return self.dropout(out)

class Block(nn.Module):
    """Transformer block with pre-norm and council MoE FFN"""
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = CouncilLayer(n_embd)  # Full council integration
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# ===========================================================
# QUILLAN GPT LANGUAGE MODEL
# ===========================================================
class QuillanGPT(nn.Module):
    """Enhanced transformer with full 32-member council MoE"""
    def __init__(self):
        super().__init__()
        # Ensure vocab_size is not 0 before creating embedding table
        if vocab_size == 0:
             raise ValueError("Vocabulary size is 0. Cannot initialize embedding table.")
        self.token_embedding_table = nn.Embedding(vocab_size, cfg.n_embd)
        self.position_embedding_table = nn.Embedding(cfg.block_size, cfg.n_embd)
        self.blocks = nn.Sequential(*[Block(cfg.n_embd, cfg.n_head) for _ in range(cfg.n_layer)])
        self.ln_f = nn.LayerNorm(cfg.n_embd)
        self.lm_head = nn.Linear(cfg.n_embd, vocab_size)

        # Initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=cfg.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=0.65, top_k=None):
        """Generate text with improved sampling"""
        for _ in range(max_new_tokens):
            # Crop context to block size
            idx_cond = idx if idx.size(1) <= cfg.block_size else idx[:, -cfg.block_size:]

            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature

            # Top-k sampling
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

# ===========================================================
# FULL COUNCIL MOE (Custom Neural Architecture for Demo)
# ===========================================================
class Value:
    """Autodiff value for council neural net - Fixed indentation"""
    def __init__(self, data: float, _children: tuple = (), _op: str = '', label: str = ''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data:.6f}, grad={self.grad:.6f})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other):
        return self * other

    def __radd__(self, other):
        return self + other

    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = Value(self.data ** other, (self,), f'**{other}')
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * Value(-1.0)

    def __sub__(self, other):
        return self + (-other)

    def __truediv__(self, other):
        return self * (other ** -1)

    def tanh(self):
        n = self.data
        t = (np.exp(2*n) - 1) / (np.exp(2*n) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def relu(self):
        out = Value(max(0, self.data), (self,), 'ReLU')
        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

class Neuron:
    """Single neuron with activation"""
    def __init__(self, nin: int, activation: str = 'tanh'):
        self.w = [Value(np.random.randn()) for _ in range(nin)]
        self.b = Value(np.random.randn())
        self.activation = activation

    def __call__(self, x: List[Value]) -> Value:
        act_input = self.b
        for wi, xi in zip(self.w, x):
            act_input = act_input + (wi * xi)

        if self.activation == 'tanh':
            return act_input.tanh()
        elif self.activation == 'relu':
            return act_input.relu()
        return act_input

    def parameters(self):
        return self.w + [self.b]

class Layer:
    """Layer of neurons"""
    def __init__(self, nin: int, nout: int, activation: str = 'tanh'):
        self.neurons = [Neuron(nin, activation) for _ in range(nout)]

    def __call__(self, x: List[Value]):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

class ExpertMLP:
    """Council expert neural network"""
    def __init__(self, nin: int, layers: List[int], activations: Optional[List[str]] = None):
        if activations is None:
            activations = ['relu'] * len(layers)
        self.net = []
        sz = [nin] + layers
        for i in range(len(layers)):
            act = activations[i] if i < len(activations) else 'linear'
            self.net.append(Layer(sz[i], sz[i+1], act))

    def __call__(self, x):
        for layer in self.net:
            x = layer(x)
            if not isinstance(x, list):
                x = [x]
        return x

    def parameters(self):
        return [p for l in self.net for p in l.parameters()]

class CouncilGating:
    """Gating mechanism for expert selection - Fixed Value softmax"""
    def __init__(self, nin, expert_count):
        self.weights = [Value(np.random.randn()) for _ in range(nin)]
        self.biases = [Value(np.random.randn()) for _ in range(expert_count)]
        self.expert_count = expert_count

    def __call__(self, x):
        logit = []
        for b in self.biases:
            weighted_sum = b
            for w, xi in zip(self.weights, x):
                weighted_sum = weighted_sum + (w * xi)
            logit.append(weighted_sum)

        # Fixed: Pure Value softmax for autodiff
        max_l = max([l.data for l in logit])
        exp_terms = [Value(np.exp(l.data - max_l)) for l in logit]
        sum_exp = sum(exp_terms)
        probs = [exp_t / sum_exp for exp_t in exp_terms]
        return probs

    def parameters(self):
        return self.weights + self.biases

class CouncilMoE:
    """Hierarchical Mixture-of-Experts council block - Fixed for Value input"""
    def __init__(self, nin, nout, n_experts=6, expert_layers=None, expert_acts=None):
        if expert_layers is None:
            expert_layers = [8, nout]
        if expert_acts is None:
            expert_acts = ['relu', 'tanh']

        self.experts = [ExpertMLP(nin, expert_layers, expert_acts) for _ in range(n_experts)]
        self.gate = CouncilGating(nin, n_experts)
        self.n_experts = n_experts

    def __call__(self, x):
        # Fixed: Ensure x is List[Value]
        if not isinstance(x, list) or not isinstance(x[0], Value):
            x = [Value(xi) for xi in x]
        gates = self.gate(x)
        expert_outs = [self.experts[i](x) for i in range(self.n_experts)]

        merged = []
        for j in range(len(expert_outs[0])):
            outj = Value(0.0)
            for i in range(self.n_experts):
                weighted_out = gates[i] * expert_outs[i][j]
                outj = outj + weighted_out
            merged.append(outj)
        return merged

    def parameters(self):
        return sum([exp.parameters() for exp in self.experts], []) + self.gate.parameters()

class QuillanMoENet:
    """Stackable council expert architecture - Full 32-expert"""
    def __init__(self,
                 input_dim: int,
                 council_shapes: List[int],
                 expert_layers: List[int] = [64, 32, 1],  # Deeper for full
                 expert_acts: List[str] = ['relu', 'tanh', 'linear']):
        self.meta_layers = []
        nin = input_dim

        for council_size in council_shapes[:-1]:
            meta = CouncilMoE(nin, council_size, n_experts=cfg.n_council_experts,  # 32
                              expert_layers=expert_layers, expert_acts=expert_acts)
            self.meta_layers.append(meta)
            nin = council_size

        self.output_council = CouncilMoE(nin, council_shapes[-1],
                                        n_experts=cfg.n_council_experts,
                                        expert_layers=expert_layers,
                                        expert_acts=expert_acts)
        self.all_params = sum([m.parameters() for m in self.meta_layers], []) + \
                         self.output_council.parameters()

    def __call__(self, x):
        # Fixed: Type-safe input wrap
        if len(x) > 0 and isinstance(x[0], Value):
            out = x[:]  # Copy
        else:
            out = [Value(xi) for xi in x]
        for meta in self.meta_layers:
            out = meta(out)
        return self.output_council(out)

    def parameters(self):
        return self.all_params

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0.0

# ===========================================================
# TRAINING LOOP
# ===========================================================
def train_quillan_gpt():
    """Main training function for Quillan GPT"""
    model = QuillanGPT().to(cfg.device)
    param_count = sum(p.numel() for p in model.parameters())
    print(f"🧠 Model initialized: {param_count/1e6:.2f}M parameters")

    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)

    best_val_loss = float('inf')

    for step in range(cfg.max_iters):
        # Evaluation
        if step % cfg.eval_interval == 0 or step == cfg.max_iters - 1:
            losses = estimate_loss(model)
            print(f"Step {step:5d} | Train: {losses['train']:.4f} | Val: {losses['val']:.4f}")

            # Save best model
            if losses['val'] < best_val_loss:
                best_val_loss = losses['val']
                torch.save({
                    'step': step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'val_loss': best_val_loss,
                }, 'quillan_best.pt')

        # Training step
        xb, yb = get_batch('train')
        logits, loss = model(xb, yb)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)

        optimizer.step()

    return model

def demo_council_moe(epochs=150):
    """Fixed demo using custom Value-based training for full council XOR"""
    print("\n" + "="*80)
    print("QUILLAN COUNCIL MOE DEMO: XOR Problem with 32 Experts")
    print("="*80)

    X = [[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]]
    Y = [[0.0], [1.0], [1.0], [0.0]]

    net = QuillanMoENet(input_dim=2, council_shapes=[32, 16, 1])  # Full hierarchy

    for epoch in range(epochs):
        total_loss = Value(0.0)
        for xi, yi in zip(X, Y):
            x_vals = [Value(v) for v in xi]
            outs = net(x_vals)
            diff = outs[0] - Value(yi[0])  # Single output
            total_loss = total_loss + (diff * diff)

        avg_loss = total_loss.data / len(X)

        net.zero_grad()
        total_loss.backward()

        for p in net.parameters():
            p.data -= 0.05 * p.grad  # SGD lr

        if epoch % 30 == 0 or epoch == epochs - 1:
            print(f"Epoch {epoch:3d} | Loss: {avg_loss:.6f}")

    # Test predictions
    print("\n📊 Final Predictions (Full Council Routing):")
    for x, y_true in zip(X, Y):
        x_vals = [Value(v) for v in x]
        y_pred = net(x_vals)[0].data
        print(f"Input: {x} | Target: {y_true[0]} | Prediction: {y_pred:.4f}")

def generate_sample_text(model, prompt="Quillan: ", max_tokens=300, temperature=0.65, top_k=40):
    """Fixed generation for QuillanGPT with thinking logs"""
    # Dramatic thinking simulation
    print("\n🧠 Quillan v4.2 COGNITIVE PROCESSING INITIATED...")
    print("[████████░░] 40% // Council activation")
    print("Activating 32-member council: C1-VIR (ethics) to C32-AEON (game dev)")
    print("Micro-swarms deployed: 224,000 agents for vector decomposition...")
    print("[███████████████░░] 75% // Deliberation")
    print("Tree of Thoughts: 20 branches explored - Selected: Hierarchical MoE fusion")
    print("[████████████████████] 100% // Ready")

    class DummyTokenizer:
        def encode(self, s):
            return encode(s)
        def decode(self, l):
            return decode(l)

    tokenizer = DummyTokenizer()

    context = torch.tensor([tokenizer.encode(prompt)], dtype=torch.long, device=cfg.device)
    generated = model.generate(context, max_new_tokens=max_tokens, temperature=temperature, top_k=top_k)
    output = tokenizer.decode(generated[0].tolist())

    print(f"\n🧠 Quillan Output:")
    print(output)
    print(f"{'='*80}\n")

    return output

def save_model(model, optimizer, step, loss, filename='quillan_checkpoint.pt'):
    """Save model checkpoint"""
    checkpoint = {
        'step': step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'config': {
            'vocab_size': vocab_size,
            'block_size': cfg.block_size,
            'n_embd': cfg.n_embd,
            'n_head': cfg.n_head,
            'n_layer': cfg.n_layer,
        }
    }
    torch.save(checkpoint, filename)
    print(f"✅ Model saved to {filename}")

def load_model(filename='quillan_checkpoint.pt'):
    """Load model checkpoint"""
    if not os.path.exists(filename):
        print(f"⚠️  Checkpoint {filename} not found")
        return None, None, 0

    checkpoint = torch.load(filename, map_location=cfg.device)
    model = QuillanGPT().to(cfg.device)
    model.load_state_dict(checkpoint['model_state_dict'])

    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    step = checkpoint['step']
    print(f"✅ Model loaded from {filename} (step {step})")

    return model, optimizer, step

# ===========================================================
# INTEGRATED EXECUTION PIPELINE
# ===========================================================

if __name__ == "__main__":
    print(f"\n{'='*80}")
    print("QUILLAN v4.2 - FULL INTEGRATED TRAINING PIPELINE")
    print(f"{'='*80}\n")

    # ============================================================
    # PHASE 1: TRANSFORMER TRAINING
    # ============================================================
    print("🔥 PHASE 1: Training Quillan GPT Transformer with Council MoE")
    print(f"{'─'*80}")

    try:
        model = train_quillan_gpt()
        print("\n✅ Transformer training complete!")

        # Generate samples with thinking
        generate_sample_text(model, prompt="Quillan: Analyze quantum entanglement implications.")

        # Save final model
        optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)
        save_model(model, optimizer, cfg.max_iters, 0.0, 'quillan_final.pt')

    except KeyboardInterrupt:
        print("\n⚠️  Training interrupted by user")
    except Exception as e:
        print(f"\n❌ Training error: {e}")
        import traceback
        traceback.print_exc()

    # ============================================================
    # PHASE 2: FULL COUNCIL MOE DEMONSTRATION
    # ============================================================
    print(f"\n{'='*80}")
    print("🧠 PHASE 2: Full 32-Member Council MoE Demo")
    print(f"{'─'*80}\n")

    try:
        demo_council_moe(epochs=150)
        print("\n✅ Full Council MoE demo complete!")

    except Exception as e:
        print(f"\n❌ Council MoE error: {e}")
        import traceback
        traceback.print_exc()

    # ============================================================
    # PHASE 3: ARCHITECTURE ANALYSIS
    # ============================================================
    print(f"\n{'='*80}")
    print("📊 PHASE 3: Full Architecture Analysis")
    print(f"{'='*80}\n")

    try:
        # Transformer stats
        if 'model' in locals():
            total_params = sum(p.numel() for p in model.parameters())
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

            print("🔹 Quillan GPT Transformer with Council MoE:")
            print(f"   Total parameters: {total_params:,}")
            print(f"   Trainable parameters: {trainable_params:,}")
            print(f"   Model size: {total_params * 4 / 1024 / 1024:.2f} MB (FP32)")
            print(f"   Layers: {cfg.n_layer}")
            print(f"   Embedding dim: {cfg.n_embd}")
            print(f"   Attention heads: {cfg.n_head}")
            print(f"   Council experts: {cfg.n_council_experts}")
            print(f"   Context window: {cfg.block_size}")
            print(f"   Vocabulary: {vocab_size} tokens")

        print("\n🔹 Full Council MoE Architecture:")
        print(f"   Council layers: {cfg.council_layers}")
        print(f"   Experts per layer: {cfg.n_council_experts}")
        print(f"   Expert hidden layers: {cfg.expert_hidden}")
        print(f"   Total experts: {cfg.n_council_experts * len(cfg.council_layers)}")

    except Exception as e:
        print(f"⚠️  Analysis error: {e}")

    # ============================================================
    # PHASE 4: INTERACTIVE MODE (Optional)
    # ============================================================
    print(f"\n{'='*80}")
    print("🎮 INTERACTIVE MODE - Full Quillan v4.2 Ready")
    print(f"{'='*80}")
    print("Options:")
    print("  1. Generate more text with council thinking")
    print("  2. Load saved checkpoint")
    print("  3. Export model")
    print("  4. Exit")
    print(f"{'─'*80}\n")

    # Note: For non-interactive environments (Colab, scripts),
    # this section can be commented out or modified

    print("✅ Quillan v4.2 FULL initialization complete!")
    print("\n🚀 System ready for deployment - 32 Council Members Active\n")

# Test model outputs:
test models outputs over 10 questions phd level or better.

In [ ]:
# Define a list of 10 PhD-level questions
phd_questions = [
    "Discuss the implications of quantum entanglement on classical information theory.",
    "Analyze the role of epigenetics in the development of complex diseases.",
    "Evaluate the effectiveness of different deep learning architectures for natural language processing tasks.",
    "Explore the ethical considerations of advanced artificial intelligence systems.",
    "Examine the impact of climate change on biodiversity in tropical ecosystems.",
    "Investigate the mechanisms of neuroplasticity and their relevance to learning and memory.",
    "Discuss the challenges and opportunities in developing sustainable energy technologies.",
    "Analyze the socio-economic factors influencing healthcare access in developing countries.",
    "Evaluate the use of blockchain technology for secure data management in distributed systems.",
    "Explore the theoretical foundations of consciousness from a computational perspective."
]

# Iterate through the questions and generate responses
for i, question in enumerate(phd_questions):
    print(f"\n--- Question {i+1}: {question} ---")
    # Encode the question as a prompt
    context = torch.tensor([encode(question)], dtype=torch.long, device=cfg.device)
    # Generate text from the model
    generated_idxs = model.generate(context, max_new_tokens=500) # Generate up to 500 new tokens
    generated_text = decode(generated_idxs[0].tolist())
    print(generated_text)
    print("-" * 20)

# Save model:
save the model in the format you want to use .pt, .gguf, .safetensor,ect...

Below are a few options.

In [ ]:
# This cell saves the model in .pt, .safetensor, and .gguf formats.

import torch
import os

# Ensure the model and optimizer are defined and trained
if 'model' not in locals() or 'optimizer' not in locals():
    print("⚠️ Model or optimizer not found. Please ensure the training cells were run.")
else:
    # 1. Save as .pt
    try:
        pt_filename = "quillan_v4_2_final.pt"
        torch.save(model.state_dict(), pt_filename)
        print(f"✅ Model saved as {pt_filename}")
    except Exception as e:
        print(f"❌ Error saving .pt model: {e}")

    # 2. Save as .safetensor
    try:
        # Requires the 'safetensors' library
        # !pip install safetensors
        from safetensors.torch import save_file

        safetensor_filename = "quillan_v4_2_final.safetensors"
        save_file(model.state_dict(), safetensor_filename)
        print(f"✅ Model saved as {safetensor_filename}")
    except ImportError:
        print("⚠️ 'safetensors' library not found. Skipping .safetensor save.")
        print("Please run: !pip install safetensors")
    except Exception as e:
        print(f"❌ Error saving .safetensor model: {e}")

    # 3. Save as .gguf
    try:
        # This requires converting to a format supported by GGUF tools (e.g., Hugging Face Transformers)
        # and then using a GGUF conversion tool (like those in the llama.cpp ecosystem).
        # The conversion process can be complex and often requires specific model architectures
        # and additional libraries (e.g., transformers, ctranslate2, llama-cpp-python).
        # A direct conversion from a custom PyTorch model state_dict to GGUF is not straightforward
        # without a defined conversion script or tool for this specific architecture.

        print("\nℹ️  GGUF conversion is complex and depends on the model architecture.")
        print("A direct conversion from this custom PyTorch model to GGUF requires a specific converter.")
        print("Skipping automatic .gguf save for now.")
        print("Please refer to GGUF conversion tools (e.g., llama.cpp) for custom models.")

        # Placeholder for potential future GGUF conversion code
        # Example (conceptual, depends on your model's compatibility):
        # from transformers import AutoModelForCausalLM, AutoTokenizer
        # # Assuming your model can be loaded by transformers
        # hf_model = AutoModelForCausalLM.from_pretrained("your_model_path", state_dict=model.state_dict())
        # hf_model.save_pretrained("./hf_model_dir")
        # # Then use a tool like llama.cpp's convert.py to create the GGUF file from ./hf_model_dir

    except Exception as e:
        print(f"❌ An error occurred during .gguf conversion attempt: {e}")